In [1]:
from tqdm.notebook import tqdm
from helpers.signals import DisableSignals

In [2]:
company_name = "CCR - Rio Sp"

In [3]:
company = Company.objects.get(name=company_name)

## ATENÇÃO

O funcionamento correto desse notebook depende total de você estar com os bancos de dados configurados da forma correta. Pra isso aqui funcionar direito, o banco "default" tem que apontar para o banco do ambiente HOMOLOG-CCR, e o banco "prod_ccr" tem que apontar para o banco do ambiente PROD-CCR. 

Você vai precisar configurar isso, não é uma configuração que vem por padrão no projeto, pois normalmente nós não acessamos dois bancos ao mesmo tempo.

Se você não sabe o que é isso, provavelmente esse notebook não vai funcionar. **Não execute se não estiver plenamente ciente do que estiver fazendo.**

Copiar custom_options e metadata da unidade de HOMOLOG para PROD

In [4]:
prod_company = Company.objects.using("ccr_prod").get(name=company_name)
prod_company.custom_options = company.custom_options
prod_company.metadata = company.metadata
prod_company.save()

OperationalError: could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?


Copiar formulários da unidade de HOMOLOG para PROD

**Atenção**: se você estiver copiando um formulário que ainda não existe em PROD, depois de executar isso aqui, você ainda vai precisar associar o formulário à unidade, criando um OccurrenceTypeSpecs

In [6]:
for a in tqdm(
    OccurrenceType.objects.filter(
        company=company,
        name__in=['Cadastro Inicial OAE','Monitoração de Iluminação']
        # insiram mais filtros caso necessário
    )
):
    a.save(using="ccr_prod")

  0%|          | 0/2 [00:00<?, ?it/s]

Copiar rodovias da unidade de HOMOLOG para PROD

**Atenção**: se você estiver copiando uma rodovia que ainda não existe em PROD, depois de executar isso aqui, você ainda vai precisar associar a rodovia à unidade

In [11]:
for a in tqdm(
    Road.objects.filter(
        company=company
        # insiram mais filtros caso necessário
    )
):
    with DisableSignals():
        a.save(using="ccr_prod")

  0%|          | 0/9 [00:00<?, ?it/s]

Copiar apontamentos da unidade de HOMOLOG para PROD

**Atenção**: se você executar isso aqui, os apontamentos ficarão sem serial. Depois você precisa rodar outra rotina para setar os seriais no novo ambiente.

In [9]:
for a in tqdm(
    Reporting.objects.filter(
        company=company, 
        # insiram mais filtros aqui pra buscar só os apontamentos que vcs quiserem
    ).values_list(
        "uuid", 
        flat=True
    )
):
    rep_obj = Reporting.objects.get(uuid=a)
    with DisableSignals():
        rep_obj.number = ""
        rep_obj.save(using="ccr_prod")

  0%|          | 0/10 [00:00<?, ?it/s]